### Notebook to perform Zernike polynominal analysis of SKA1-LOW and LOFAR calibration

In [ ]:
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
#pylab.rcParams['image.cmap'] = 'rainbow'

from see import see

from telutil import *

import subprocess
print "Working from Git repository %s" % subprocess.check_output(["git",  "describe", "--long", "--all"])

def trials(ntrials,config,name,rmin=0,rmax=200.0,tiono=10,tobs=10000*3600.0,snrthreshold=5.0):
    tp=TelPiercings()

    random.seed(781490893)

    # Minimum SNR=10 per pierce point
    for trial in range(ntrials):
        ts=TelSources()
        ts.construct(nsources=nsources, radius=wavelength/stationdiameter)


        tp.construct(ts,config,hiono=hiono,rmin=rmin,rmax=HWZ)
        if trial==0:
            tp.plot(rcore=HWZ, rmax=2.0*HWZ)

        s=numpy.sqrt(tp.assess(nnoll=nnoll, rmax=HWZ, doplot=(trial==0)))
        if trial==0:
            Save=s/float(ntrials)
        else:
            Save=Save+s/float(ntrials)
    J=len(Save[Save>snrthreshold])
    DR=TelIono().dr(J,tobs=tobs)
    tobs=TelIono().tobs(J,DR=1e5)

    return {'dr':DR, 's':Save, 'J':J, 'tobs':tobs}
            
def plotstats(stats):
    plt.clf()
    colors={'LOWBD2':'b', 'LOWBD2-CORE':'g', 'LOWBD2-HALO':'r', 'LOWBD2-RASTERHALO':'c', \
            'LOWBD2-RASTERHALO25km':'purple', 'LOWBD2-RASTER':'m', 'LOFAR':'k'}
    for name in stats.keys():
        plt.semilogy(range(len(stats[name]['s'])), stats[name]['s'], color=colors[name], label=name)
    plt.xlabel('Singular vector index')
    plt.ylabel('Singular value')
    plt.legend(loc='upper right')
    plt.title('Singular values for %d source(s)' % nsources)
    maxs=nsources*nnoll
    plt.axes().set_xlim([1,nnoll])
    plt.axes().set_ylim([0.1,1e4])
    name='SingularValues_sources%d' % nsources
    plt.show()
    plt.savefig('%s.pdf' % name)

def printstats(s, nsources):
    for key in stats.keys():
        print "Nsources %d, Array %s peak=%.1f, J=%d, DR=%.1f (dB), tobs=%.1f (years), MST=%.1f (km)" \
        % (nsources, key, stats[key]['s'][0], stats[key]['J'], \
        10.0*numpy.log10(stats[key]['dr']), stats[key]['tobs']/(24.0*3600.0*365.0), stats[key]['MST'])

ntrials=10
rhalo=40.0
nsources=70
nnoll=1000
wavelength=3.0
stationdiameter=35.0
rcore=0.5
rmin=0.0
rmax=40.0
hiono=300.0
HWZ=hiono*wavelength/stationdiameter # Half width to Zero
print "FOV (HWZ) at ionosphere=%.1f km" % HWZ

### Look at all arrays

In [ ]:
for nsources in [1, 3, 10, 30]:
    ts=TelSources()
    random.seed(781490893)
    ts.construct(nsources=nsources, radius=wavelength/stationdiameter)
    stats={}
    lowbd=TelArray()
    lowbd.readKML(kmlfile='V4Drsst512red_2.kml')
    lowbd.saveCSV('LOWBD2.csv')
    lowbd.plot()
    stats['LOWBD2']=trials(ntrials, lowbd, 'LOWBD2')
    stats['LOWBD2']['MST'] = lowbd.mst()
    plotstats(stats)
    printstats(stats, nsources)
    lowcore=TelArray()
    lowcore.readCSV('LOWBD2-CORE', l1def='LOWBD2-CORE.csv',recenter=True)
    lowcore.plot()
    stats['LOWBD2-CORE']=trials(ntrials, lowcore, 'LOWCORE')
    stats['LOWBD2-CORE']['MST'] = lowcore.mst()
    plotstats(stats)
    printstats(stats, nsources)
    lowhalo=TelArray()
    lowhalo.readCSV('LOWBD2-HALO', l1def='LOWBD2-HALO.csv')
    lowhalo.plot()
    stats['LOWBD2-HALO']=trials(ntrials, lowhalo, 'LOWBD2-HALO')
    stats['LOWBD2-HALO']['MST'] = lowhalo.mst()
    plotstats(stats)
    printstats(stats, nsources)
    lowhalo=TelArray()
    lowhalo.rasterBoolardy(nhalo=346, name='LOWRASTERHALO', nstations=346, scale=1.0, \
                           rhalo=40.0, rcore=0.0)
    lowcore=TelArray()
    lowcore.readCSV('LOWCORE', l1def='LOWBD2-CORE.csv',recenter=True)
    lowraster=TelArray().add(lowhalo,lowcore,name='LOWBD2-RASTERHALO')
    lowraster.plot()
    stats['LOWBD2-RASTERHALO']=trials(ntrials, lowraster, 'LOWBD2-RASTERHALO')
    stats['LOWBD2-RASTERHALO']['MST'] = lowraster.mst()

    lowhalo=TelArray()
    lowhalo.rasterBoolardy(nhalo=346, name='LOWRASTERHALO25km', nstations=346, scale=40.0/25.7, \
                           rhalo=25.7, rcore=0.0)
    lowcore=TelArray()
    lowcore.readCSV('LOWCORE', l1def='LOWBD2-CORE.csv',recenter=True)
    lowraster=TelArray().add(lowhalo,lowcore,name='LOWBD2-RASTERHALO25km')
    lowraster.plot()
    stats['LOWBD2-RASTERHALO25km']=trials(ntrials, lowraster, 'LOWBD2-RASTERHALO25km')
    stats['LOWBD2-RASTERHALO25km']['MST'] = lowraster.mst()

    plotstats(stats)
    printstats(stats, nsources)
    lofar=TelArray()
    lofar.readLOFAR()
    lofar.plot()
    stats['LOFAR']=trials(ntrials, lofar, 'LOFAR')
    stats['LOFAR']['MST'] = lofar.mst()
    plotstats(stats)
    printstats(stats, nsources)

